# Installing the library

In [ ]:
# !git clone https://github.com/Guzpenha/transformer_rankers.git
%cd /content/transformer_rankers 
!git checkout full_rank_retrieval_dialogues
! rm -rf env
! pip install virtualenv
! virtualenv env
!source env/bin/activate
! pip install -e .
! sed -i '/pandas/d' ./requirements.txt 
! sed -i '/numpy/d' ./requirements.txt 
! pip install -r requirements.txt

/content/transformer_rankers
M	requirements.txt
Already on 'full_rank_retrieval_dialogues'
Your branch is up to date with 'origin/full_rank_retrieval_dialogues'.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
created virtual environment CPython3.7.15.final.0-64 in 187ms
  creator CPython3Posix(dest=/content/transformer_rankers/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==22.3, setuptools==65.5.0, wheel==0.37.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/transformer_rankers
  Attempting uninstall: transformer-rankers
    Found existing installation: transformer-rankers 0.0.1
  

In [ ]:
#Install Anserini which is also a requirement for part of transformer-rankers (BM25 Negative Samplers)
!apt-get install maven -qq
!git clone --recurse-submodules https://github.com/castorini/anserini.git
!cd anserini; mvn clean package appassembler:assemble -DskipTests -Dmaven.javadoc.skip=true
!ls anserini/target/appassembler/bin/

Extracting templates from packages: 100%
Selecting previously unselected package libapache-pom-java.
(Reading database ... 123942 files and directories currently installed.)
Preparing to unpack .../00-libapache-pom-java_18-1_all.deb ...
Unpacking libapache-pom-java (18-1) ...
Selecting previously unselected package libatinject-jsr330-api-java.
Preparing to unpack .../01-libatinject-jsr330-api-java_1.0+ds1-5_all.deb ...
Unpacking libatinject-jsr330-api-java (1.0+ds1-5) ...
Selecting previously unselected package libgeronimo-interceptor-3.0-spec-java.
Preparing to unpack .../02-libgeronimo-interceptor-3.0-spec-java_1.0.1-4fakesync_all.deb ...
Unpacking libgeronimo-interceptor-3.0-spec-java (1.0.1-4fakesync) ...
Selecting previously unselected package libcdi-api-java.
Preparing to unpack .../03-libcdi-api-java_1.2-2_all.deb ...
Unpacking libcdi-api-java (1.2-2) ...
Selecting previously unselected package libcommons-cli-java.
Preparing to unpack .../04-libcommons-cli-java_1.4-1_all.deb ...

# Downloading data

In [ ]:
import logging
import pandas as pd
logging.basicConfig(level=logging.INFO,  format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[logging.StreamHandler()])

In [ ]:
from transformer_rankers.datasets import downloader

task = 'mantis'
data_folder = "./data/"
logging.info("Starting downloader for task {}".format(task))

dataDownloader = downloader.DataDownloader(task, data_folder)
dataDownloader.download_and_preprocess()

[INFO] Please confirm you agree to the authors' data usage agreement found at <https://ciir.cs.umass.edu/downloads/Antique/readme.txt>
[INFO] If you have a local copy of https://ciir.cs.umass.edu/downloads/Antique/antique-collection.txt, you can symlink it here to avoid downloading it again: /root/.ir_datasets/downloads/684f7015aff377062a758e478476aac8
[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/antique-collection.txt
[INFO] [finished] https://ciir.cs.umass.edu/downloads/Antique/antique-collection.txt: [00:11] [93.6MB] [8.00MB/s]
[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/antique-train-queries.txt
[INFO] [finished] https://ciir.cs.umass.edu/downloads/Antique/antique-train-queries.txt: [00:00] [137kB] [191kB/s]
[INFO] [starting] https://ciir.cs.umass.edu/downloads/Antique/antique-train.qrel
[INFO] [finished] https://ciir.cs.umass.edu/downloads/Antique/antique-train.qrel: [00:01] [626kB] [525kB/s]
[INFO] [starting] https://ciir.cs.umass.edu/downloads/

In [ ]:
train = pd.read_csv("./data/{}/train.tsv".format(task), sep="\t")
train.head()

,query,passage,rel
0,Why do some men spit into the urinal before ur...,I do it all the time. It is kind of a ritual ...,4
1,Why do some men spit into the urinal before ur...,To clear out the mucus deep down in the throat...,4
2,Why do some men spit into the urinal before ur...,"maybe they want a target to hit. Well, I gues...",3
3,Why do some men spit into the urinal before ur...,Where else would we spit?... Apart from sports...,3
4,Why do some men spit into the urinal before ur...,Because they have a cough or phlegm and hacked...,3


In [ ]:
valid = pd.read_csv(data_folder+task+"/valid.tsv", sep="\t")
valid = valid[:100] #sampling so that eval is faster
valid.head()

,query,passage,rel
0,Why do teens today find it amusing to tear one...,OMG! How did you get through high school? Te...,4
1,Why do teens today find it amusing to tear one...,the rumors part is girls only.i have never hea...,2
2,Why do teens today find it amusing to tear one...,"its not teens today, teens have always been li...",3
3,Why do teens today find it amusing to tear one...,Because their parents don't give them enough a...,3
4,Why do teens today find it amusing to tear one...,cuz its funny,2


# Finetunning pointwise BERT (monoBERT) for MANtIS with early stop after 1000 instances

By BERT here we consider monoBERT [1], the ranking approach that uses a classification head on top of the [CLS] token and fine-tunes BERT with cross-entropy loss in a pointwise manner. The dialogue context is concatenated with special tokens [UTTERANCE_SEP] and [TURN_SEP].

*\[1] LIN, Jimmy; NOGUEIRA, Rodrigo; YATES, Andrew. Pretrained transformers for text ranking: Bert and beyond. 2020.*

In [ ]:
from transformers import BertTokenizerFast

from transformer_rankers.models import pointwise_bert
from transformer_rankers.trainers import transformer_trainer
from transformer_rankers.datasets import dataset, preprocess_crr
from transformer_rankers.negative_samplers import negative_sampling
from transformer_rankers.eval import results_analyses_tools
from transformer_rankers.models import pointwise_bert

# Random negative samplers
ns_train = negative_sampling.BM25NegativeSamplerPyserini(list(train["response"].values), 1, data_folder+task+"/anserini_train/", -1, "./anserini/")
negative_samples_test = 9
ns_val = negative_sampling.BM25NegativeSamplerPyserini(list(valid["response"].values) + \
    list(train["response"].values), negative_samples_test, data_folder+task+"/anserini_train/", -1, "./anserini/")

tokenizer = BertTokenizerFast.from_pretrained('bert-base-cased')
special_tokens_dict = {'additional_special_tokens': ['[UTTERANCE_SEP]', '[TURN_SEP]'] }
tokenizer.add_special_tokens(special_tokens_dict)
      
#Create the loaders for the datasets, with the respective negative samplers        
dataloader = dataset.QueryDocumentDataLoader(train_df=train, val_df=valid, test_df=valid,
                                tokenizer=tokenizer, negative_sampler_train=ns_train, 
                                negative_sampler_val=ns_val, task_type='classification', 
                                train_batch_size=32, val_batch_size=32, max_seq_len=100, 
                                sample_data=-1, cache_path="{}/{}".format(data_folder, task)) 

train_loader, val_loader, test_loader = dataloader.get_pytorch_dataloaders()

model = pointwise_bert.BertForPointwiseLearning.from_pretrained('bert-base-cased')
model.resize_token_embeddings(len(dataloader.tokenizer))

#Instantiate trainer that handles fitting.
trainer = transformer_trainer.TransformerTrainer(model=model,train_loader=train_loader,
                                val_loader=val_loader, test_loader=test_loader, 
                                num_ns_eval=negative_samples_test, task_type="classification", tokenizer=tokenizer,
                                validate_every_epochs=1, num_validation_batches=-1,
                                num_epochs=1, lr=0.0005, sacred_ex=None, 
                                validate_every_steps=-1, num_training_instances=10000)

#Train the model
logging.info("Fitting monoBERT for {}".format(task))
trainer.fit()

#Predict for test (in our example the validation set)
logging.info("Predicting")
preds, labels, softmax = trainer.test()
res = results_analyses_tools.\
    evaluate_and_aggregate(preds, labels, ['R_10@1', 'R_10@5', 'map'])

for metric, v in res.items():
    logging.info("Test {} : {:4f}".format(metric, v))

ImportError: ignored

The test effectiveness is actually based on a small sample (100) of the validation set (see code).